In [1]:
import io
import os
import requests
import pandas as pd
from google.cloud import storage
import urllib.request
import pyarrow

In [2]:
# services = ['fhv','green','yellow']
init_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/'

# switch out the bucketname
BUCKET = os.environ.get("GCP_GCS_BUCKET", "dtc_data_lake_ny-rides-kelvin")

In [3]:
def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    # # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # # (Ref: https://github.com/googleapis/python-storage/issues/74)
    # storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    # storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB

    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [ ]:
def web_to_gcs(year, service):
    for i in range(12):
        i = i + 1
        
        # sets the month part of the file_name string
        month = '0'+str(i+1)
        month = month[-2:]

        # csv.gz  file_name 
        file_name = service + '_tripdata_' + year + '-' + month + '.csv.gz'
        
        # download csv.gz file
        request_url = init_url + file_name
        urllib.request.urlretrieve(request_url, '../homework/data/' + file_name)

        df = pd.read_csv('../homework/data/' + file_name, encoding='latin1')
        file_name = file_name.replace('.csv.gz', '.parquet')
        df.to_parquet('../homework/data/' + file_name, engine='pyarrow')
        print(f"Parquet: {file_name}")

        # upload it to gcs 
        upload_to_gcs(BUCKET, f"{service}_parquet/{file_name}", '../homework/data/' + file_name)
        print(f"GCS: {service}_parquet/{file_name}")


# web_to_gcs('2019', 'green')
# web_to_gcs('2020', 'green')
# web_to_gcs('2019', 'yellow')
# web_to_gcs('2020', 'yellow')
# web_to_gcs('2019', 'fhv')
web_to_gcs('2020', 'fhv')
# web_to_gcs('2021', 'fhv')

In [4]:
pwd

'/Users/kelvinye/Documents/de-zoomcamp/week_4/2023/ingest'

In [14]:
file_name = 'fhv' + '_tripdata_' + '2020' + '-' + '01' + '.csv.gz'
df = pd.read_csv('../data/' + file_name)
# df.shape[0]
df.tail(10)

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
2028447,B03257,2020-01-31 23:58:06,2020-02-01 00:15:44,NaN,76.0,NaN,B02682
2028448,B03266,2020-01-31 23:19:01,2020-01-31 23:22:21,179.0,7.0,NaN,B03266
2028449,B03266,2020-01-31 23:45:28,2020-01-31 23:52:44,179.0,179.0,NaN,B03266
2028450,B03266,2020-01-31 23:09:17,2020-01-31 23:17:48,223.0,129.0,NaN,B03266
2028451,B03266,2020-01-31 23:09:23,2020-01-31 23:16:39,193.0,7.0,NaN,B03266
2028452,B03266,2020-01-31 23:39:08,2020-01-31 23:46:24,223.0,223.0,NaN,B03266
2028453,B03266,2020-01-31 23:51:11,2020-02-01 00:09:02,193.0,56.0,NaN,B03266
2028454,B03266,2020-01-31 23:12:40,2020-01-31 23:19:56,7.0,7.0,NaN,B03266
2028455,B03266,2020-01-31 23:28:37,2020-01-31 23:35:53,7.0,7.0,NaN,B03266
2028456,B03266,2020-01-31 23:19:09,2020-01-31 23:26:25,223.0,223.0,NaN,B03266


In [18]:
file_name = 'yellow' + '_tripdata_' + '2019' + '-' + '07' + '.parquet'
df = pd.read_parquet('../data/' + file_name)
# df.shape[0]
df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2019-07-01 00:51:04,2019-07-01 00:51:33,1.0,0.00,1.0,N,193,193,1.0,2.5,0.5,0.5,1.14,0.00,0.3,4.94,0.0
1,2.0,2019-07-01 00:46:04,2019-07-01 01:05:46,1.0,4.16,1.0,N,234,25,2.0,16.5,0.5,0.5,0.00,0.00,0.3,20.30,2.5
2,1.0,2019-07-01 00:25:09,2019-07-01 01:00:56,1.0,18.80,2.0,N,132,42,1.0,52.0,0.0,0.5,11.75,6.12,0.3,70.67,0.0
3,2.0,2019-07-01 00:33:32,2019-07-01 01:15:27,1.0,18.46,2.0,N,132,142,1.0,52.0,0.0,0.5,11.06,0.00,0.3,66.36,2.5
4,1.0,2019-07-01 00:00:55,2019-07-01 00:13:05,0.0,1.70,1.0,N,107,114,1.0,9.5,3.0,0.5,2.00,0.00,0.3,15.30,2.5
5,1.0,2019-07-01 00:18:23,2019-07-01 00:30:42,0.0,2.10,1.0,N,125,45,1.0,10.0,3.0,0.5,2.75,0.00,0.3,16.55,2.5
6,1.0,2019-07-01 00:28:09,2019-07-01 00:51:00,1.0,2.40,1.0,N,142,68,1.0,13.0,3.0,0.5,3.35,0.00,0.3,20.15,2.5
7,1.0,2019-07-01 00:57:07,2019-07-01 01:11:41,1.0,3.00,1.0,N,246,141,2.0,12.5,3.0,0.5,0.00,0.00,0.3,16.30,2.5
8,4.0,2019-07-01 00:06:16,2019-07-01 00:33:14,1.0,7.89,1.0,N,50,80,1.0,26.0,0.5,0.5,5.96,0.00,0.3,35.76,2.5
9,4.0,2019-07-01 00:37:19,2019-07-01 00:52:30,1.0,4.09,1.0,N,80,97,2.0,15.0,0.5,0.5,0.00,0.00,0.3,16.30,0.0


In [19]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object